In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
ON_CPU = device == torch.device('cpu')

In [ ]:
DIR_INPUT = '/kaggle/input/smartathon-dataset-zip/dataset'
train = pd.read_csv(f'{DIR_INPUT}/train.csv')
test = pd.read_csv(f'{DIR_INPUT}/test.csv')
test.shape

In [ ]:
test['xmin'] = 0
test['ymin'] = 0
test['xmax'] = 1
test['ymax'] = 1
test['class'] = 0
test['name'] = 'NA'

In [ ]:
image_ids = test['image_path'].unique()

test_ids = image_ids[:]

test_df = test[test['image_path'].isin(test_ids)].copy()

In [ ]:
class CreateDataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_path'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_path'] == image_id]

        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        area = torch.as_tensor(area, dtype=torch.float32)

        labels = records['class']
        labels = labels.to_numpy(dtype= np.int64)
        labels = torch.from_numpy(labels)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = torch.from_numpy(boxes)
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# Albumentations
def get_test_transform():
    return A.Compose([
        A.Resize(960, 540),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        A.Resize(960, 540),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
def get_model():
    """
    https://stackoverflow.com/questions/58362892/resnet-18-as-backbone-in-faster-r-cnn
    """
    backbone = resnet_fpn_backbone('resnet101', pretrained=True)
    model = FasterRCNN(backbone, num_classes=11)
    return model

In [ ]:
import os
models_list = os.listdir('/kaggle/input/fork-of-faster-rcnn-smartathon')
models_list.remove('__results__.html')
models_list.remove('__notebook__.ipynb')
models_list.remove('__output__.json')
models_list.remove('custom.css')

In [ ]:
loaded_models = []

for model_path in models_list:
    model = get_model()
    path = '/kaggle/input/fork-of-faster-rcnn-smartathon' + '/' + model_path
    model.load_state_dict(torch.load(path,
                                     map_location=torch.device('cpu')
                                    )
                         )
    loaded_models.append(model)

In [ ]:
CLASSES = train['class'].unique()
CLASSES = list(CLASSES)
#CLASSES.append('__background__')
#CLASSES
num_classes = len(CLASSES)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

DIR_TRAIN = '/kaggle/input/smartathon-dataset-zip/dataset/images'
test_dataset = CreateDataset(test_df, DIR_TRAIN,get_test_transform())

indices = torch.randperm(len(test_dataset)).tolist()

test_data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    collate_fn=collate_fn
)

In [ ]:
# model = get_model()
# model.to(device)
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
# # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
# lr_scheduler = None

# num_epochs = 23

In [ ]:
preds = []


classes = train['class'].unique()
names = train['name'].unique()
classmap = {class_val : name for (class_val, name) in zip(classes,names)}

In [ ]:
for model in loaded_models:
    model.eval()
    model.to(device)
    output_dict = {}
    output_dict['class']= []
    output_dict['image_path'] = []
    output_dict['name'] = []
    output_dict['xmin'] = []
    output_dict['xmax'] = []
    output_dict['ymin'] = []
    output_dict['ymax'] = []

    idx = 0

    for images, targets, image_ids in test_data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        outputs = model(images)
        boxes = outputs[0]['boxes'].cpu().detach().numpy()
        labels =  outputs[0]['labels'].cpu().detach().numpy()

        for pred in range(boxes.shape[0]):
            (xmin, xmax, ymin, ymax) = boxes[pred]
            label = labels[pred]
            output_dict['class'].append(label)
            output_dict['image_path'].append(test['image_path'][idx])
            output_dict['name'].append('NA')
            output_dict['xmin'].append(xmin)
            output_dict['xmax'].append(xmax)
            output_dict['ymin'].append(ymin)
            output_dict['ymax'].append(ymax)

        if idx%500 == 499:
            print(idx+1, 'completed')
        idx += 1
        
    data = pd.DataFrame(output_dict)
    data['name'] = data['class'].map(classmap)
    print(len(test['image_path'].unique()) - len(data['image_path'].unique()))
    print(set(test['image_path'].unique()) - set(data['image_path'].unique()))
    preds.append(data) 

In [ ]:
for i in range(len(preds)):
    preds[i].to_csv(f'/kaggle/working/frcnn101_epoch{2*i}.csv')

In [ ]:
for i in range(len(preds[11:])):
    append_list = set(test['image_path'].unique()) - set(preds[11+i]['image_path'].unique())
    print(len(preds[11+i]))
    for path in append_list:
        row = {'image_path':path}
        preds[11+i] = preds[11+i].append(row, ignore_index = True)
    print(len(preds[11+i]))
    print()
    preds[11+i].to_csv(f'/kaggle/working/frcnn101_epoch{2*i}.csv')